In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
#taking state code and state name mapping from c16 file

c16="DDW-C16-STMT-MDDS-0000.XLSX"
df_c16=pd.read_excel(c16,skiprows=[0,1,3,4,5])
df_c16 = df_c16[["State","Area name"]]
codes= dict(zip(df_c16["State"],df_c16["Area name"]))
codes

{0: 'INDIA',
 1: 'JAMMU & KASHMIR',
 2: 'HIMACHAL PRADESH',
 3: 'PUNJAB',
 4: 'CHANDIGARH',
 5: 'UTTARAKHAND',
 6: 'HARYANA',
 7: 'NCT OF DELHI',
 8: 'RAJASTHAN',
 9: 'UTTAR PRADESH',
 10: 'BIHAR',
 11: 'SIKKIM',
 12: 'ARUNACHAL PRADESH',
 13: 'NAGALAND',
 14: 'MANIPUR',
 15: 'MIZORAM',
 16: 'TRIPURA',
 17: 'MEGHALAYA',
 18: 'ASSAM',
 19: 'WEST BENGAL',
 20: 'JHARKHAND',
 21: 'ODISHA',
 22: 'CHHATTISGARH',
 23: 'MADHYA PRADESH',
 24: 'GUJARAT',
 25: 'DAMAN & DIU',
 26: 'DADRA & NAGAR HAVELI',
 27: 'MAHARASHTRA',
 28: 'ANDHRA PRADESH',
 29: 'KARNATAKA',
 30: 'GOA',
 31: 'LAKSHADWEEP',
 32: 'KERALA',
 33: 'TAMIL NADU',
 34: 'PUDUCHERRY',
 35: 'ANDAMAN & NICOBAR ISLANDS'}

In [3]:
#Taking the individual state files in C17 folder, processing them for Mother tongue and MotherTongue+2ndlang+3rdLang speakers
#append all state data to one dataframe df_c17_final
df_c17_final=pd.DataFrame()

for filename in os.listdir("C17"):
    c17=os.path.join("C17",filename)
    col_names=['State Code','State','L_Code.1','L_Name.1','Persons.1','Males.1','Females.1','L_Code.2','L_Name.2','Persons.2','Males.2','Females.2','L_Code.3','L_Name.3','Persons.3','Males.3','Females.3']
    df_c17=pd.read_excel(c17,names=col_names,skiprows=5)
    
    df_L1=df_c17[['L_Code.1','L_Name.1','Persons.1']]
    df_L2=df_c17[['L_Code.2','L_Name.2','Persons.2']]
    df_L3=df_c17[['L_Code.3','L_Name.3','Persons.3']]

    df_L1.dropna(inplace=True)
    df_L1.reset_index(drop=True,inplace=True)

    df_L2.dropna(inplace=True)
    df_L2=df_L2.groupby(['L_Code.2','L_Name.2']).agg({'Persons.2':'sum'}).reset_index()

    df_L3.dropna(inplace=True)
    df_L3=df_L3.groupby(['L_Code.3','L_Name.3']).agg({'Persons.3':'sum'}).reset_index()

    df_L = pd.merge(df_L1,df_L2, how= "left",left_on=['L_Code.1','L_Name.1'],right_on = ['L_Code.2','L_Name.2'])
    df_L.drop(['L_Name.2','L_Code.2'],axis=1,inplace=True)
    df_L=pd.merge(df_L,df_L3, how= "left",left_on=['L_Code.1','L_Name.1'],right_on = ['L_Code.3','L_Name.3'])
    df_L.drop(['L_Name.3','L_Code.3'],axis=1,inplace=True)

    df_L['LanguageSpeakers']=df_L['Persons.1']+df_L['Persons.2']+df_L['Persons.3']
    df_L.rename(columns={'Persons.1':'MotherTongue','L_Code.1':'L_Code','L_Name.1':'L_Name',},inplace=True)
    df_L.drop(['Persons.2','Persons.3'],axis=1,inplace=True)

    new_col = df_c17['State Code']  
    df_L.insert(loc=0, column='State Code', value=new_col)
    new_col = df_c17['State'] 
    df_L.insert(loc=1, column='State', value=new_col)
    df_c17_final=pd.concat([df_c17_final,df_L])
df_c17_final.reset_index(drop=True,inplace=True)

In [4]:
"""North: JK, Ladakh, PN, HP, HR, UK, Delhi, Chandigarh 
   West: RJ, GJ, MH, Goa, Dadra & Nagar Haveli, Daman & Diu 
   Central: MP, UP, CG 
   East: BH, WB, OR, JH 
   South: KT, TG, AP, TN, KL, Lakshadweep, Puducherry 
   North-East: AS, SK, MG, TR, AR, MN, NG, MZ, Andaman & Nicobar"""

"""{0: 'INDIA', 1: 'JAMMU & KASHMIR', 2: 'HIMACHAL PRADESH', 3: 'PUNJAB', 4: 'CHANDIGARH', 5: 'UTTARAKHAND', 6: 'HARYANA',
 7: 'NCT OF DELHI',8: 'RAJASTHAN',9: 'UTTAR PRADESH',10: 'BIHAR',11: 'SIKKIM',12: 'ARUNACHAL PRADESH',13: 'NAGALAND',
 14: 'MANIPUR',15: 'MIZORAM',16: 'TRIPURA',17: 'MEGHALAYA',18: 'ASSAM',19: 'WEST BENGAL',20: 'JHARKHAND',21: 'ODISHA',
 22: 'CHHATTISGARH',23: 'MADHYA PRADESH',24: 'GUJARAT',25: 'DAMAN & DIU',26: 'DADRA & NAGAR HAVELI',27: 'MAHARASHTRA',
 28: 'ANDHRA PRADESH',29: 'KARNATAKA',30: 'GOA',31: 'LAKSHADWEEP',32: 'KERALA',33: 'TAMIL NADU',34: 'PUDUCHERRY',
 35: 'ANDAMAN & NICOBAR ISLANDS'}"""

#assign State codes to regions based on above mapping

north=[1,2,3,4,5,6,7]
west=[8,24,25,26,27,30]
central=[23,9,22]
east=[10,19,21,20]
south=[29,28,33,32,31,34]
north_east=[18,11,17,16,12,14,13,15,35]


In [5]:
#separate the dataframes based on State codes into regions 
df_c17_north=df_c17_final.loc[df_c17_final['State Code'].isin(north)]
df_c17_west=df_c17_final.loc[df_c17_final['State Code'].isin(west)]
df_c17_central=df_c17_final.loc[df_c17_final['State Code'].isin(central)]
df_c17_east=df_c17_final.loc[df_c17_final['State Code'].isin(east)]
df_c17_south=df_c17_final.loc[df_c17_final['State Code'].isin(south)]
df_c17_north_east=df_c17_final.loc[df_c17_final['State Code'].isin(north_east)]

In [6]:
#take all dataframes in a tupple and calculate MT and Language speakers for each state.
#Append each state data to different lists.
dfs = (df_c17_north,df_c17_west,df_c17_central,df_c17_east,df_c17_south,df_c17_north_east)
dfs_mt= []
dfs_tot=[]
for df in dfs:
    df_test=df.groupby("L_Name").agg({'MotherTongue':'sum','LanguageSpeakers':'sum'}).reset_index()
    df_test.sort_values(by='MotherTongue',ascending=False,inplace=True,ignore_index=True)
    mt = df_test["L_Name"][:3]
    dfs_mt.append(mt)
    df_test.sort_values(by='LanguageSpeakers',ascending=False,inplace=True,ignore_index=True)
    tot = df_test["L_Name"][:3]
    dfs_tot.append(tot)

In [7]:
#convert above mother tongue list to data frame
final_mt= pd.DataFrame(dfs_mt)
final_mt.columns=["language-1","language-2","language-3"]
new_col = ["North","West","Central","East","South","North East"] 
final_mt.insert(loc=0, column='region', value=new_col)
final_mt.reset_index(drop=True,inplace=True)
final_mt

,region,language-1,language-2,language-3
0,North,HINDI,PUNJABI,KASHMIRI
1,West,HINDI,MARATHI,GUJARATI
2,Central,HINDI,URDU,BHILI/BHILODI
3,East,HINDI,BENGALI,ODIA
4,South,TELUGU,TAMIL,KANNADA
5,North East,ASSAMESE,BENGALI,HINDI


In [8]:
#convert above total language speakers list to data frames 
final_tot=pd.DataFrame(dfs_tot)
final_tot.columns=["language-1","language-2","language-3"]
new_col = ["North","West","Central","East","South","North East"] 
final_tot.insert(loc=0, column='region', value=new_col)
final_tot.reset_index(drop=True,inplace=True)
final_tot

,region,language-1,language-2,language-3
0,North,HINDI,PUNJABI,ENGLISH
1,West,HINDI,MARATHI,GUJARATI
2,Central,HINDI,ENGLISH,URDU
3,East,HINDI,BENGALI,ODIA
4,South,TELUGU,TAMIL,KANNADA
5,North East,ASSAMESE,BENGALI,HINDI


In [9]:
#load to csv
outname_1 = 'region-india-a.csv'
outname_2 = 'region-india-b.csv'
outfiles = './Output_Files'
if not os.path.exists(outfiles):
    os.mkdir(outfiles)
outdir=outfiles+'/Q7'
if not os.path.exists(outdir):
    os.mkdir(outdir)
filepath_1 = os.path.join(outdir, outname_1)  
filepath_2 = os.path.join(outdir, outname_2)

final_mt.to_csv(filepath_1,index=False)
final_tot.to_csv(filepath_2,index=False)